In [1]:
import argparse
import json
import os
import os.path as osp
import sys

import torch
from mmcv import Config

from dataset import build_data_loader
from models import build_model
from models.utils import fuse_module
from utils import AverageMeter, Corrector, ResultFormat, Visualizer

/opt/conda/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
cfg = Config.fromfile('config/pan_pp/pan_pp_test.py')
# for d in [cfg, cfg.data.test]:
#     d.update(dict(report_speed=args.report_speed))
# cfg.update(dict(vis=args.vis))
# cfg.update(dict(debug=args.debug))
# cfg.data.test.update(dict(debug=args.debug))
print(json.dumps(cfg._cfg_dict, indent=4))

{
    "model": {
        "type": "PAN_PP",
        "backbone": {
            "type": "resnet50",
            "pretrained": true
        },
        "neck": {
            "type": "FPEM_v2",
            "in_channels": [
                256,
                512,
                1024,
                2048
            ],
            "out_channels": 128
        },
        "detection_head": {
            "type": "PAN_PP_DetHead",
            "in_channels": 512,
            "hidden_dim": 128,
            "num_classes": 6,
            "loss_text": {
                "type": "DiceLoss",
                "loss_weight": 1.0
            },
            "loss_kernel": {
                "type": "DiceLoss",
                "loss_weight": 0.5
            },
            "loss_emb": {
                "type": "EmbLoss_v2",
                "feature_dim": 4,
                "loss_weight": 0.25
            },
            "use_coordconv": false
        }
    },
    "data": {
        "batch_size": 1,
        "trai

In [3]:
cfg.data.test

{'type': 'PAN_PP_TEST',
 'split': 'test',
 'short_size': 1024,
 'read_type': 'cv2',
 'with_rec': False}

In [4]:
data_loader = build_data_loader(cfg.data.test)
test_loader = torch.utils.data.DataLoader(
    data_loader,
    batch_size=1,
    shuffle=False,
    num_workers=2,
)

reading type: cv2.


In [5]:
if hasattr(cfg.model, 'recognition_head'):
    cfg.model.recognition_head.update(
        dict(
            voc=data_loader.voc,
            char2id=data_loader.char2id,
            id2char=data_loader.id2char,
        ))
model = build_model(cfg.model)
model = model.cuda()

In [6]:
checkpoint = './weights/doc_panpp_best_weight.pth.tar'

if checkpoint is not None:
    if os.path.isfile(checkpoint):
        print("Loading model and optimizer from checkpoint '{}'".format(
            checkpoint))

        checkpoint = torch.load(checkpoint)

        d = dict()
        for key, value in checkpoint['state_dict'].items():
            tmp = key[7:]
            d[tmp] = value
        model.load_state_dict(d)
    else:
        print("No checkpoint found at '{}'".format(args.resume))
        raise

Loading model and optimizer from checkpoint './weights/doc_panpp_best_weight.pth.tar'


In [7]:
model = fuse_module(model)

def model_structure(model):
    blank = ' '
    print('-' * 90)
    print('|' + ' ' * 11 + 'weight name' + ' ' * 10 + '|' \
          + ' ' * 15 + 'weight shape' + ' ' * 15 + '|' \
          + ' ' * 3 + 'number' + ' ' * 3 + '|')
    print('-' * 90)
    num_para = 0

    for index, (key, w_variable) in enumerate(model.named_parameters()):
        if len(key) <= 30:
            key = key + (30 - len(key)) * blank
        shape = str(w_variable.shape)
        if len(shape) <= 40:
            shape = shape + (40 - len(shape)) * blank
        each_para = 1
        for k in w_variable.shape:
            each_para *= k
        num_para += each_para
        str_num = str(each_para)
        if len(str_num) <= 10:
            str_num = str_num + (10 - len(str_num)) * blank

        print('| {} | {} | {} |'.format(key, shape, str_num))
    print('-' * 90)
    print('The total number of parameters: ' + str(num_para))
    print('The parameters of Model {}: {:4f}M'.format(
        model._get_name(), num_para / 1e6))
    print('-' * 90)
    

model_structure(model)

------------------------------------------------------------------------------------------
|           weight name          |               weight shape               |   number   |
------------------------------------------------------------------------------------------
| backbone.conv1.weight          | torch.Size([64, 3, 3, 3])                | 1728       |
| backbone.conv1.bias            | torch.Size([64])                         | 64         |
| backbone.conv2.weight          | torch.Size([64, 64, 3, 3])               | 36864      |
| backbone.conv2.bias            | torch.Size([64])                         | 64         |
| backbone.conv3.weight          | torch.Size([128, 64, 3, 3])              | 73728      |
| backbone.conv3.bias            | torch.Size([128])                        | 128        |
| backbone.layer1.0.conv1.weight | torch.Size([64, 128, 1, 1])              | 8192       |
| backbone.layer1.0.conv1.bias   | torch.Size([64])                         | 64         |

In [8]:
model.eval()

with_rec = hasattr(cfg.model, 'recognition_head')

rf = ResultFormat(cfg.data.test.type, cfg.test_cfg.result_path)
print('Start testing %d images' % len(test_loader))
cfg.debug = False
cfg.report_speed = False

for idx, data in enumerate(test_loader):
    print('Testing %d/%d\r' % (idx, len(test_loader)), end='', flush=True)
    
    # prepare input
    data['imgs'] = data['imgs'].cuda()
    data.update(dict(cfg=cfg))

    # forward
    with torch.no_grad():
        outputs = model(**data)

    # save result
    image_name, _ = osp.splitext(osp.basename(test_loader.dataset.img_paths[idx]))
    image_path = test_loader.dataset.img_paths[idx]
    rf.write_result(image_name, image_path, outputs)
#     rf.write_result(data['img_metas'], outputs)

print('Clear')

Start testing 101 images


/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:3060: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn("Default upsampling behavior when mode={} is changed "



Testing 1/101
Testing 2/101
Testing 3/101
Testing 4/101
Testing 5/101
Testing 6/101
Testing 7/101
Testing 8/101
Testing 9/101
Testing 10/101
Testing 11/101
Testing 12/101
Testing 13/101
Testing 14/101
Testing 15/101
Testing 16/101
Testing 17/101
Testing 18/101
Testing 19/101
Testing 20/101
Testing 21/101
Testing 22/101
Testing 23/101
Testing 24/101
Testing 25/101
Testing 26/101
Testing 27/101
Testing 28/101
Testing 29/101
Testing 30/101
Testing 31/101
Testing 32/101
Testing 33/101
Testing 34/101
Testing 35/101
Testing 36/101
Testing 37/101
Testing 38/101
Testing 39/101
Testing 40/101
Testing 41/101
Testing 42/101
Testing 43/101
Testing 44/101
Testing 45/101
Testing 46/101
Testing 47/101
Testing 48/101
Testing 49/101
Testing 50/101
Testing 51/101
Testing 52/101
Testing 53/101
Testing 54/101
Testing 55/101
Testing 56/101
Testing 57/101
Testing 58/101
Testing 59/101
Testing 60/101
Testing 61/101
Testing 62/101
Testing 63/101
Testing 64/101
Testing 65/101
Testing 66/101
Testing 67/101
Tes